In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 顶点客户端库：TF Hub 图像分类模型用于在线预测

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_tfhub_image_classification_online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_tfhub_image_classification_online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Python的Vertex客户端库，将预训练的TensorFlow Hub图像分类模型部署到在线预测中。

数据集

本教程使用的数据集是来自[TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview)的[花卉数据集](https://www.tensorflow.org/datasets/catalog/tf_flowers)。在本教程中使用的数据集版本存储在一个公共云存储桶中。训练过的模型可以预测一张图像所属的花卉类型，包括五种花卉：雏菊、蒲公英、玫瑰、向日葵或郁金香。

### 目标

在本教程中，您将部署一个TensorFlow Hub预训练模型，然后通过发送数据对部署的模型进行预测。

执行的步骤包括：

- 下载一个TensorFlow Hub预训练模型。
- 检索并加载模型的构件。
- 将模型上传为Vertex `Model`资源。
- 部署`Model`资源到一个服务`Endpoint`资源。
- 进行预测。
- 取消部署`Model`资源。

费用

本教程使用谷歌云(GCP)的可计费组件：

* Vertex AI
* 云存储

了解[Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)和[云存储价格](https://cloud.google.com/storage/pricing)，并使用[Pricing计算器](https://cloud.google.com/products/calculator/)根据您预计的使用量生成费用估算。

安装

安装最新版本的Vertex客户端库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install -U google-cloud-aiplatform $USER_FLAG

安装最新的GA版本*google-cloud-storage*库。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

### 重新启动内核

一旦您安装了Vertex客户端库和Google *cloud-storage*，您需要重新启动笔记本内核，以便它可以找到这些包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

*如果您的选项中有 GPU 运行时，请确保在该笔记本中运行。在 Colab 中，选择* **运行时 > 修改运行时类型 > GPU**

### 设置您的谷歌云项目

**无论您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。第一次创建帐户时，您将获得 $300 的免费信用额用于您的计算/存储成本。

2. [确保为您的项目启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用 Vertex API 和 Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)。

4. [谷歌云 SDK](https://cloud.google.com/sdk) 已经安装在谷歌云笔记本中。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，以确保云 SDK 在本笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter 运行以 `!` 开头的行作为 shell 命令，并将以 `$` 开头的 Python 变量插入这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

地区

您也可以更改 `REGION` 变量，该变量用于整个笔记本的操作。以下是 Vertex 支持的地区。我们建议您选择最靠近您的地区。

- 美洲: `us-central1`
- 欧洲: `europe-west4`
- 亚太地区: `asia-east1`

您可能无法使用多区域存储桶进行 Vertex 训练。并非所有地区都提供对所有 Vertex 服务的支持。有关每个地区的最新支持，请参阅[Vertex 位置文档](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在进行实时教程会话，您可能会使用共享的测试帐户或项目。为了避免用户在创建的资源之间发生名称冲突，您为每个实例会话创建一个时间戳，并附加到将在此教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的Google Cloud账号

**如果您正在使用Google Cloud Notebook**，您的环境已经通过验证。请跳过此步骤。

**如果您正在使用Colab**，运行下面的单元格，并按提示进行身份验证以通过oAuth验证您的账户。

**否则**，请按照以下步骤操作：

在Cloud控制台中，转到[创建服务账号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务账号**。

在**服务账号名称**字段中输入一个名称，然后点击**创建**。

在**授予此服务账号对项目的访问权限**部分，点击角色下拉框。在过滤框中输入“Vertex”，并选择**Vertex管理员**。在过滤框中输入“Storage Object Admin”，并选择**存储对象管理员**。

点击创建。包含您密钥的JSON文件将下载到本地环境中。

在下面的单元格中，将您的服务账号密钥的路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用什么笔记本环境，以下步骤都是必需的。**

当您使用 Vertex 客户端库提交一个自定义训练作业时，您需要上传一个包含训练代码的 Python 包到一个云存储桶中。Vertex 会从这个包中运行代码。在本教程中，Vertex 还会将您的作业生成的训练模型保存在同一个存储桶中。接着，您可以基于这个输出创建一个`Endpoint`资源，以便提供在线预测。

在下面设置您的云存储桶的名称。存储桶的名称必须在所有 Google Cloud 项目中是全局唯一的，包括您组织外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有在您的存储桶不存在时才运行以下单元格来创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查云存储桶的内容来验证对其的访问。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

#### 导入顶点客户端库

将顶点客户端库导入到我们的Python环境中。

In [ ]:
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

顶点常量

为顶点设置以下常量：

- `API_ENDPOINT`：用于数据集、模型、作业、流水线和端点服务的顶点API服务端点。
- `PARENT`：用于数据集、模型、作业、流水线和端点资源的顶点位置根路径。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

#### 硬件加速器

为预测设置硬件加速器（例如，GPU）。

将变量`DEPLOY_GPU/DEPLOY_NGPU`设置为使用支持GPU的容器映像以及分配给虚拟机（VM）实例的GPU数量。例如，要将使用4个Nvidia Telsa K80 GPU分配给每个VM的GPU容器图像，您可以指定：

    （aip.AcceleratorType.NVIDIA_TESLA_K80, 4）

对于GPU，可用的加速器包括：
   - aip.AcceleratorType.NVIDIA_TESLA_K80
   - aip.AcceleratorType.NVIDIA_TESLA_P100
   - aip.AcceleratorType.NVIDIA_TESLA_P4
   - aip.AcceleratorType.NVIDIA_TESLA_T4
   - aip.AcceleratorType.NVIDIA_TESLA_V100

否则，指定`(None, None)`来使用在CPU上运行的容器映像。

In [ ]:
if os.getenv("IS_TESTING_DEPOLY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPOLY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

### 容器（Docker）映像

接下来，我们将设置用于预测的Docker容器映像

- 将变量`TF`设置为容器映像的TensorFlow版本。例如，`2-1`将是版本2.1，`1-15`将是版本1.15。以下列表显示了一些预构建的可用映像:

 - TensorFlow 1.15
   - `gcr.io/cloud-aiplatform/prediction/tf-cpu.1-15:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf-gpu.1-15:latest`
 - TensorFlow 2.1
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-1:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-1:latest`
 - TensorFlow 2.2
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-2:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-2:latest`
 - TensorFlow 2.3
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-3:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-3:latest`
 - XGBoost
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-2:latest`
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-1:latest`
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.0-90:latest`
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.0-82:latest`
 - Scikit-learn
   - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-23:latest`
   - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-22:latest`
   - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-20:latest`

有关最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers)

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2-1"

if TF[0] == "2":
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

DEPLOY_IMAGE = "gcr.io/cloud-aiplatform/prediction/{}:latest".format(DEPLOY_VERSION)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU)

#### 机器类型

接下来，设置用于预测的机器类型。

- 将变量`DEPLOY_COMPUTE`设置为配置要用于预测的VM的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB的内存。
     - `n1-highmem`：每个vCPU 6.5GB的内存。
     - `n1-highcpu`：每个vCPU 0.9GB的内存。
 - `vCPUs`：数量为\[2，4，8，16，32，64，96\]

*注意：您还可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

# 教程

现在您已准备好部署一个TensorFlow Hub预训练图像分类模型。

## 设置客户端

Vertex 客户端库工作在客户端/服务器模型中。在您的一侧 (Python 脚本)，您将创建一个客户端，发送请求并从 Vertex 服务器接收响应。

在本教程中，您将针对工作流程中不同步骤使用不同的客户端。因此，请提前设置它们。

- 用于 `Model` 资源的模型服务。
- 用于部署的端点服务。
- 用于提供服务的预测服务。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(client_options=client_options)
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(client_options=client_options)
    return client


clients = {}
clients["model"] = create_model_client()
clients["endpoint"] = create_endpoint_client()
clients["prediction"] = create_prediction_client()

for client in clients.items():
    print(client)

从TFHub获取预训练模型

接下来，您可以从$(TENSORFLOW) Hub下载一个预训练模型。

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

IMAGE_SHAPE = (224, 224)

model = tf.keras.Sequential(
    [
        hub.KerasLayer(
            "https://tfhub.dev/google/imagenet/resnet_v1_50/feature_vector/4",
            input_shape=IMAGE_SHAPE + (3,),
        )
    ]
)

model_path_to_deploy = BUCKET_NAME + "/resnet"

## 上传模型进行提供服务

接下来，您将从定制作业中将您的 TF.Keras 模型上传到 Vertex `Model` 服务中，这将为您的定制模型创建一个 Vertex `Model` 资源。在上传过程中，您需要定义一个服务函数，将数据转换为模型期望的格式。如果您将编码数据发送到 Vertex，您的服务函数将确保数据在传递到模型之前在模型服务器上解码。

### 服务函数如何工作

当您向在线预测服务器发送请求时，请求将由 HTTP 服务器接收。HTTP 服务器从 HTTP 请求内容主体中提取预测请求。提取的预测请求被转发到服务函数。对于 Google 预构建的预测容器，请求内容将作为 `tf.string` 传递到服务函数。

服务函数由两部分组成：

- `预处理函数`：
  - 将输入 (`tf.string`) 转换为底层模型的输入形状和数据类型 (动态图)。
  - 执行与训练底层模型期间相同的数据预处理 -- 例如，标准化，缩放等。
- `后处理函数`：
  - 将模型输出转换为接收应用程序期望的格式 -- 例如，压缩输出。
  - 为接收应用程序打包输出 -- 例如，添加标题，创建 JSON 对象等。

预处理和后处理函数都被转换为静态图，这些静态图与模型融合在一起。底层模型的输出传递给后处理函数。后处理函数将转换/打包后的输出传递回HTTP服务器。HTTP 服务器将输出作为 HTTP 响应内容返回。

在构建 TF.Keras 模型的服务函数时，您需要考虑一个因素，即它们作为静态图运行。这意味着您不能使用需要动态图的 TF 图操作。如果这样做，您将在服务函数的编译过程中收到一个错误，该错误将指示您正在使用不受支持的 EagerTensor。

### 图像数据的服务功能

为了将图像传递给预测服务，您需要将压缩（例如JPEG）图像字节编码为base64格式--这样可以使内容在通过网络传输二进制数据时安全免受修改。由于该部署模型期望输入数据为原始（未压缩）字节，您需要确保base64编码数据在传递给部署模型之前被转换回原始字节。

为了解决这个问题，定义一个服务函数（`serving_fn`）并将其附加到模型作为预处理步骤。添加一个`@tf.function`装饰器，以便服务函数融合到底层模型中（而不是在CPU上游）。

当您发送预测或解释请求时，请求的内容会被base64解码为一个Tensorflow字符串（`tf.string`），然后传递给服务函数（`serving_fn`）。服务函数将`tf.string`预处理为原始（未压缩）的numpy字节（`preprocess_fn`）以匹配模型的输入要求：
- `io.decode_jpeg`- 解压缩JPG图像，返回一个带有三个通道（RGB）的Tensorflow张量。
- `image.convert_image_dtype` - 将整数像素值更改为float32。
- `image.resize` - 将图像调整大小以匹配模型的输入形状。
- `resized / 255.0` - 将像素数据重新缩放（规范化）到0和1之间。

在这一点上，数据可以传递给模型（`m_call`）。

In [ ]:
CONCRETE_INPUT = "numpy_inputs"


def _preprocess(bytes_input):
    decoded = tf.io.decode_jpeg(bytes_input, channels=3)
    decoded = tf.image.convert_image_dtype(decoded, tf.float32)
    resized = tf.image.resize(decoded, size=(32, 32))
    rescale = tf.cast(resized / 255.0, tf.float32)
    return rescale


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def preprocess_fn(bytes_inputs):
    decoded_images = tf.map_fn(
        _preprocess, bytes_inputs, dtype=tf.float32, back_prop=False
    )
    return {
        CONCRETE_INPUT: decoded_images
    }  # User needs to make sure the key matches model's input


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def serving_fn(bytes_inputs):
    images = preprocess_fn(bytes_inputs)
    prob = m_call(**images)
    return prob


m_call = tf.function(model.call).get_concrete_function(
    [tf.TensorSpec(shape=[None, 32, 32, 3], dtype=tf.float32, name=CONCRETE_INPUT)]
)

tf.saved_model.save(
    model, model_path_to_deploy, signatures={"serving_default": serving_fn}
)

获取服务功能签名

您可以通过重新加载模型并查询与每个层对应的签名来获取模型的输入和输出层的签名。

对于您的目的，您需要服务功能的签名。为什么呢？当我们将数据作为HTTP请求数据包发送进行预测时，图像数据将被base64编码，而我们的TF.Keras模型接受numpy输入。您的服务功能将从base64转换为numpy数组。

在进行预测请求时，您需要将请求路由到服务功能而不是模型，因此您需要知道服务功能的输入层名称 -- 这将在以后进行预测请求时使用。

In [ ]:
loaded = tf.saved_model.load(model_path_to_deploy)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)

### 上传模型

使用这个辅助函数`upload_model`来上传您的模型，存储在SavedModel格式中，到`Model`服务，该服务将为您的模型实例化一个Vertex`Model`资源实例。一旦您完成了这个步骤，您可以像任何其他Vertex`Model`资源实例一样使用`Model`资源实例，例如部署到`Endpoint`资源以提供预测。

辅助函数接受以下参数：

- `display_name`：`Endpoint`服务的人类可读名称。
- `image_uri`：模型部署的容器镜像。
- `model_uri`：我们的SavedModel艺术品在Cloud Storage中的路径。对于本教程，这是`trainer/task.py`保存模型工件的Cloud Storage位置，在该路径中我们指定了变量`MODEL_DIR`。

辅助函数调用`Model`客户端服务的方法`upload_model`，该方法接受以下参数：

- `parent`：`Dataset`、`Model`和`Endpoint`资源的Vertex根路径。
- `model`：Vertex`Model`资源实例的规范。

现在让我们深入了解Vertex模型规范`model`。这是一个包含以下字段的字典对象：

- `display_name`：`Model`资源的人类可读名称。
- `metadata_schema_uri`：由于您的模型是在没有Vertex`Dataset`资源的情况下构建的，因此将此字段留空（''）。
- `artificat_uri`：模型存储在SavedModel格式中的Cloud Storage路径。
- `container_spec`：这是将安装在`Endpoint`资源上的Docker容器的规范，从中`Model`资源将提供预测。使用您之前设置的变量`DEPLOY_GPU != None`来使用GPU；否则只分配CPU。

将模型上传到Vertex Model资源将返回一个长时间运行的操作，因为可能需要一些时间。您调用`response.result()`，这是一个同步调用，在Vertex Model资源准备就绪时返回。

辅助函数返回对应的Vertex Model实例`upload_model_response.model`的Vertex完全限定标识符。您将保存这个标识符用于后续步骤中的变量`model_to_deploy_id`。

In [ ]:
IMAGE_URI = DEPLOY_IMAGE


def upload_model(display_name, image_uri, model_uri):
    model = {
        "display_name": display_name,
        "metadata_schema_uri": "",
        "artifact_uri": model_uri,
        "container_spec": {
            "image_uri": image_uri,
            "command": [],
            "args": [],
            "env": [{"name": "env_name", "value": "env_value"}],
            "ports": [{"container_port": 8080}],
            "predict_route": "",
            "health_route": "",
        },
    }
    response = clients["model"].upload_model(parent=PARENT, model=model)
    print("Long running operation:", response.operation.name)
    upload_model_response = response.result(timeout=180)
    print("upload_model_response")
    print(" model:", upload_model_response.model)
    return upload_model_response.model


model_to_deploy_id = upload_model(
    "flowers-" + TIMESTAMP, IMAGE_URI, model_path_to_deploy
)

### 获取`Model`资源信息

现在让我们仅获取您的模型的模型信息。使用这个辅助函数`get_model`，并传入以下参数：

- `name`：`Model`资源的Vertex唯一标识符。

这个辅助函数调用了Vertex `Model`客户端服务的`get_model`方法，并传入以下参数：

- `name`：`Model`资源的Vertex唯一标识符。

In [ ]:
def get_model(name):
    response = clients["model"].get_model(name=name)
    print(response)


get_model(model_to_deploy_id)

部署 `Model` 资源

现在部署训练好的 Vertex 自定义 `Model` 资源。这需要两个步骤：

1. 创建一个 `Endpoint` 资源来部署 `Model` 资源。

2. 将 `Model` 资源部署到 `Endpoint` 资源上。

### 创建`Endpoint`资源

使用这个辅助函数`create_endpoint`来创建一个用于部署模型以用于提供预测的终端节点，需要以下参数：

- `display_name`：`Endpoint`资源的易读名称。

这个辅助函数使用终端节点客户端服务的`create_endpoint`方法，该方法接受以下参数：

- `display_name`：`Endpoint`资源的易读名称。

创建一个`Endpoint`资源会返回一个长时间运行的操作，因为可能需要一些时间来为提供服务的`Endpoint`资源进行配置。您可以调用`response.result()`来同步调用，当Endpoint资源准备好时将返回。这个辅助函数返回`Endpoint`资源的 Vertex 完全限定标识符：`response.name`。

In [ ]:
ENDPOINT_NAME = "flowers_endpoint-" + TIMESTAMP


def create_endpoint(display_name):
    endpoint = {"display_name": display_name}
    response = clients["endpoint"].create_endpoint(parent=PARENT, endpoint=endpoint)
    print("Long running operation:", response.operation.name)

    result = response.result(timeout=300)
    print("result")
    print(" name:", result.name)
    print(" display_name:", result.display_name)
    print(" description:", result.description)
    print(" labels:", result.labels)
    print(" create_time:", result.create_time)
    print(" update_time:", result.update_time)
    return result


result = create_endpoint(ENDPOINT_NAME)

现在获得您创建的“终端”资源的唯一标识符。

In [ ]:
# The full unique ID for the endpoint
endpoint_id = result.name
# The short numeric ID for the endpoint
endpoint_short_id = endpoint_id.split("/")[-1]

print(endpoint_id)

### 计算实例缩放

处理在线预测请求时，您有几种选择来扩展计算实例：

- 单个实例：在线预测请求在单个计算实例上处理。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为一。

- 手动缩放：在线预测请求被分配到您手动指定的固定数量的计算实例中。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为相同的节点数。当模型首次部署到实例上时，将提供固定数量的计算实例，并将在线预测请求均匀分配给它们。

- 自动缩放：在线预测请求被分配到可扩展数量的计算实例中。
  - 将计算实例的最小（`MIN_NODES`）数量设置为在模型首次部署时提供和取消提供的实例数量，并将最大（`MAX_NODES`）数量设置为根据负载情况提供的实例数量。

计算实例的最小数量对应于字段 `min_replica_count`，而最大数量对应于字段 `max_replica_count`，在您后续的部署请求中。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

### 部署 `Model` 资源到 `Endpoint` 资源

使用此辅助函数 `deploy_model` 来将 `Model` 资源部署到您为提供预测创建的 `Endpoint` 资源，使用以下参数：

- `model`: 要从训练管道中上传（部署）的 Vertex 完全限定模型标识符。
- `deploy_model_display_name`: 部署模型的人类可读名称。
- `endpoint`: 要将模型部署到的 Vertex 完全限定的终端标识符。

辅助函数调用 `Endpoint` 客户端服务的 `deploy_model` 方法，该方法接受以下参数：

- `endpoint`: 要将 `Model` 资源部署到的 Vertex 完全限定的 `Endpoint` 资源标识符。
- `deployed_model`: 部署模型的要求规范。
- `traffic_split`: 流量分配到此模型的终端的百分比，表示为一个或多个键/值对的字典。
   - 如果只有一个模型，则指定为 **{ "0": 100 }**，其中 "0" 指的是要上传的这个模型，100 表示100% 的流量。
   - 如果终端上已经存在模型，需要将流量分配给这些模型，则使用 `model_id` 指定为 **{ "0": percent, model_id: percent, ... }**，其中 `model_id` 是要部署到的终端的现有模型的模型标识符。这些百分比必须加起来等于 100。

现在让我们深入了解 `deployed_model` 参数。此参数被指定为具有最低必需字段的 Python 字典：

- `model`: 要部署的 Vertex 完全限定模型标识符。
- `display_name`: 部署模型的人类可读名称。
- `disable_container_logging`: 这会禁用容器事件的日志记录，例如执行失败（默认情况下启用容器日志记录）。通常在部署调试时启用容器日志记录，然后在生产部署时禁用。
- `dedicated_resources`: 这指的是为提供预测请求而缩放的计算实例（副本）数量。
  - `machine_spec`: 要提供的计算实例。使用您之前设置的变量 `DEPLOY_GPU != None` 来使用 GPU；否则只分配了一个 CPU。
  - `min_replica_count`: 初始提供的计算实例数量，您之前设置为变量 `MIN_NODES`。
  - `max_replica_count`: 可扩展到的最大计算实例数量，您之前设置为变量 `MAX_NODES`。

#### 流量分配

现在让我们深入了解 `traffic_split` 参数。此参数被指定为一个 Python 字典。这可能一开始有点令人困惑。让我解释一下，您可以将多个模型实例部署到一个终端，然后设置每个模型实例所获得的流量百分比。

为什么要这样做呢？也许您已经在生产环境部署了以前的版本 —— 让我们称之为 v1。对于 v2，您得到更好的模型评估，但在最终部署到生产环境之前，您并不确定它是否真的更好。因此，在流量分配的情况下，您可能希望将 v2 部署到与 v1 相同的终端，但只将其分配给 10% 的流量。这样，您可以监视它的表现，而不会干扰大多数用户，直到做出最终决定。

#### 响应

该方法返回一个长时间运行的操作 `response`。我们将通过调用 `response.result()` 同步等待操作完成，并且会一直阻塞直到模型部署完毕。如果这是第一次将模型部署到终端，可能需要额外几分钟来完成资源的配备。

In [ ]:
DEPLOYED_NAME = "flowers_deployed-" + TIMESTAMP


def deploy_model(
    model, deployed_model_display_name, endpoint, traffic_split={"0": 100}
):

    if DEPLOY_GPU:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_type": DEPLOY_GPU,
            "accelerator_count": DEPLOY_NGPU,
        }
    else:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_count": 0,
        }

    deployed_model = {
        "model": model,
        "display_name": deployed_model_display_name,
        "dedicated_resources": {
            "min_replica_count": MIN_NODES,
            "max_replica_count": MAX_NODES,
            "machine_spec": machine_spec,
        },
        "disable_container_logging": False,
    }

    response = clients["endpoint"].deploy_model(
        endpoint=endpoint, deployed_model=deployed_model, traffic_split=traffic_split
    )

    print("Long running operation:", response.operation.name)
    result = response.result()
    print("result")
    deployed_model = result.deployed_model
    print(" deployed_model")
    print("  id:", deployed_model.id)
    print("  model:", deployed_model.model)
    print("  display_name:", deployed_model.display_name)
    print("  create_time:", deployed_model.create_time)

    return deployed_model.id


deployed_model_id = deploy_model(model_to_deploy_id, DEPLOYED_NAME, endpoint_id)

发起一个在线预测请求

现在对您部署的模型进行在线预测。

获取测试项目

您将使用数据集中的示例图像作为测试项目。

In [ ]:
FLOWERS_CSV = "gs://cloud-ml-data/img/flower_photos/all_data.csv"

test_images = ! gsutil cat $FLOWERS_CSV | head -n1
test_image = test_images[0].split(",")[0]
print(test_image)

### 准备请求内容

您将以压缩的JPG图像形式发送鲜花图像，而不是原始未压缩的字节：

- `tf.io.read_file`：将压缩的JPG图像读取回内存中作为原始字节。
- `base64.b64encode`：将原始字节编码为base64编码字符串。

In [ ]:
import base64

bytes = tf.io.read_file(test_image)
b64str = base64.b64encode(bytes.numpy()).decode("utf-8")

### 发送预测请求

好的，现在你有一张测试图片。使用这个辅助函数 `predict_image`，它接受以下参数：

- `image`：作为 numpy 数组的测试图片数据。
- `endpoint`：用于指定 `Model` 资源部署的 Vertex 完全限定标识符的 `Endpoint` 资源。
- `parameters_dict`：用于预测的额外参数。

此函数使用以下参数调用预测客户端服务 `predict` 方法：

- `endpoint`：用于指定 `Model` 资源部署的 Vertex 完全限定标识符的 `Endpoint` 资源。
- `instances`：要预测的实例（编码图像）的列表。
- `parameters`：用于预测的额外参数。

为了将图像数据传递给预测服务，在上一步中你将字节编码为 base64 格式 -- 这使得在网络上传输二进制数据时内容不易被修改。你需要告诉服务器二进制在哪里部署了你的模型，并且内容已经被 base64 编码，这样它会在服务端解码。

预测请求中的每个实例是一个形如以下的字典条目：

                        {serving_input: {'b64': content}}

- `input_name`：底层模型的输入层名称。
- `'b64'`：指示内容已经被 base64 编码的关键字。
- `content`：压缩的 JPG 图像字节作为 base64 编码的字符串。

由于 `predict()` 服务可以接受多个图像（实例），你将会把单个图像作为一个列表中唯一的图像发送。最后一步，你要把实例列表打包成谷歌的 protobuf 格式 -- 这是我们传递给 `predict()` 服务的内容。

`response` 对象返回一个列表，列表中的每个元素对应请求中的对应图像。你会在每个预测输出中看到：

- `predictions`：每个类别的预测置信度，介于 0 到 1 之间。

In [ ]:
def predict_image(image, endpoint, parameters_dict):
    # The format of each instance should conform to the deployed model's prediction input schema.
    instances_list = [{serving_input: {"b64": image}}]
    instances = [json_format.ParseDict(s, Value()) for s in instances_list]

    response = clients["prediction"].predict(
        endpoint=endpoint, instances=instances, parameters=parameters_dict
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    predictions = response.predictions
    print("predictions")
    for prediction in predictions:
        print(" prediction:", prediction)


predict_image(b64str, endpoint_id, None)

## 取消部署 `Model` 资源

现在从正在提供服务的 `Endpoint` 资源中取消部署您的 `Model` 资源。使用这个辅助函数 `undeploy_model`，它接受以下参数：

- `deployed_model_id`：当 `Model` 资源部署到 `Endpoint` 服务时由端点服务返回的模型部署标识符。
- `endpoint`：`Model` 部署到的 `Endpoint` 资源的 Vertex 完全限定标识符。

此函数调用端点客户端服务的方法 `undeploy_model`，带有以下参数：

- `deployed_model_id`：当 `Model` 资源部署时端点服务返回的模型部署标识符。
- `endpoint`：`Model` 资源部署到的 `Endpoint` 资源的 Vertex 完全限定标识符。
- `traffic_split`：如何在 `Endpoint` 资源上的其余部署模型间分配流量。

由于这是在 `Endpoint` 资源上唯一部署的模型，您可以通过将其设置为空对象 {} 来简单地不填写 `traffic_split`。

In [ ]:
def undeploy_model(deployed_model_id, endpoint):
    response = clients["endpoint"].undeploy_model(
        endpoint=endpoint, deployed_model_id=deployed_model_id, traffic_split={}
    )
    print(response)


undeploy_model(deployed_model_id, endpoint_id)

# 清理工作

要清理此项目中使用的所有GCP资源，您可以[删除](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)用于本教程的GCP项目。

否则，您可以删除本教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 终端
- 批处理作业
- 自定义作业
- 超参数调整作业
- 云存储存储桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and "dataset_id" in globals():
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and "pipeline_id" in globals():
        clients["pipeline"].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and "model_to_deploy_id" in globals():
        clients["model"].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and "endpoint_id" in globals():
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and "batch_job_id" in globals():
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and "job_id" in globals():
        clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and "hpt_job_id" in globals():
        clients["job"].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME